In [4]:
import numpy as np
from PIL import Image
import multiprocessing as mp

def is_blue_vectorized(tile, threshold=40):
    return (tile[:,:,2] > tile[:,:,0] + threshold) & (tile[:,:,2] > tile[:,:,1] + threshold)

def process_tile(args):
    img, box, threshold = args
    tile = np.array(img.crop(box))
    return np.sum(is_blue_vectorized(tile, threshold))

def calculate_blue_area(image_path, real_width_km=2, real_height_km=2, tile_size=1024, threshold=40):
    with Image.open(image_path) as img:
        width, height = img.size
        total_pixels = width * height

        tiles = [
            (img, (i, j, min(i + tile_size, width), min(j + tile_size, height)), threshold)
            for i in range(0, width, tile_size)
            for j in range(0, height, tile_size)
        ]

        with mp.Pool() as pool:
            blue_pixels = sum(pool.map(process_tile, tiles))

        blue_ratio = blue_pixels / total_pixels
        total_area_km2 = real_width_km * real_height_km
        blue_area_km2 = blue_ratio * total_area_km2

        return blue_pixels, total_pixels, blue_area_km2, total_area_km2

if __name__ == '__main__':
    image_path = '/content/78fbe741-27e7-4e75-a179-9a1bd3a1292a.jpg'  # Replace with your image path
    blue_count, total_count, blue_area, total_area = calculate_blue_area(image_path)

    print(f"Number of blue pixels: {blue_count}")
    print(f"Total pixels: {total_count}")
    print(f"Ratio of blue pixels to total pixels: {blue_count/total_count:.4f}")
    print(f"Total area: {total_area:.2f} km²")
    print(f"Blue area: {blue_area:.4f} km²")
    print(f"Percentage of blue area: {(blue_area/total_area)*100:.2f}%")

Number of blue pixels: 4562454
Total pixels: 67108864
Ratio of blue pixels to total pixels: 0.0680
Total area: 4.00 km²
Blue area: 0.2719 km²
Percentage of blue area: 6.80%
